In [10]:
import pandas as pd
import numpy as np
# Se cambio que el numero de clases ya no es parametro
# Ojo que si establecemos los bins fijos, no hay que sumar el dato minimo
def create_freq_dist_table2(data):
    min_data = data[0].min()
    min_data_ocurr = data[0].value_counts()[min_data]
    max_data = data[0].max()
    range_data = max_data - min_data
    bins = range(90,121,5)
    data['class'] = pd.cut(data[0],bins=bins)
    data = data.groupby("class")[0].count().to_frame(name="abs_freq")
    # data.iloc[0]["abs_freq"] += min_data_ocurr
    data["rel_freq"] = data.abs_freq/data.abs_freq.sum()
    data["cum_abs_freq"] = data.abs_freq.cumsum()
    data["cum_rel_freq"] = data["rel_freq"].cumsum()
    data = data.reset_index()
    data['class_mark'] = data['class'].apply(lambda x: (x.left+x.right)/2)
    # cols = data.columns.tolist()    TODO
    # cols = cols[-1:] + cols[:-1]    TODO
    return data,bins

def data_from_table(table):
    media = round(sum(table.apply(lambda x:(x['rel_freq']*x['class_mark']),axis=1)),6)
    std = round(np.sqrt(sum(table.apply(lambda row:((row['class_mark']-media)**2)*row['abs_freq'],axis = 1))/table.iloc[-1,3]),6)
    std1 = np.sqrt(sum(table.apply(lambda row:((row['class_mark']-media)**2)*row['abs_freq'],axis = 1))/(table.iloc[-1,3]-1))
    asim = sum(table.apply(lambda row:((row['class_mark']-media)**3)*row['abs_freq'],axis = 1))/(table.iloc[-1,3]*(std1**3))
    kurt = (sum(table.apply(lambda row:((row['class_mark']-media)**4)*row['abs_freq'],axis = 1))/(table.iloc[-1,3]*(std1**4)))-3
    quantile_25 = quantile(25,table)
    quantile_50 = quantile(50,table)
    quantile_75 = quantile(75,table)
    min=table.iloc[0,0].left
    max=table.iloc[-1,0].right
    return pd.Series([media,std,min,quantile_25,quantile_50,quantile_75,max,asim,kurt])

# Genera datos aleatorios, con la misma distribucion de los datos originales
def generate_data(freq_table,k):
    gen_data = pd.Series(dtype='float64')
    n_data_bef = freq_table.iloc[-1]["cum_abs_freq"]
    intervals, freq_bef = freq_table["class"], freq_table["rel_freq"]
    for i in range(len(intervals)):
        data_numb = int(round(freq_bef.iloc[i]*k,0))
        low_bound = intervals.iloc[i].left
        high_bound = intervals.iloc[i].right
        gen_data = pd.concat([gen_data,pd.Series(np.random.uniform(low_bound,high_bound,data_numb))])
    return pd.DataFrame(gen_data)
 
def quantile(q, table):
    n = max(table['cum_abs_freq'])
    index = (q/100)*n
    condition = table['cum_abs_freq'] >= index
    row =  condition.idxmax()
    Li = table['class'].iloc[row].left
    len_interval = table['class'].iloc[row].length
    Ni = table['abs_freq'].iloc[row]
    j = index - table['cum_abs_freq'].iloc[row-1]
    return (Li + len_interval*(j/Ni))

In [ ]:
def data_from_histogram(num_data, lim_inf,lim_sup,lista):
  for i in range(num_data):
    lista.append(np.random.uniform(lim_inf,lim_sup))
  return lista

lista = []
lista = data_from_histogram(2,90,95,lista)
lista = data_from_histogram(8,95,100,lista)
lista = data_from_histogram(5,100,105,lista)
lista = data_from_histogram(4,105,110,lista)
lista = data_from_histogram(6,110,115,lista)
lista = data_from_histogram(5,115,120,lista)
data2 = pd.Series(lista)
def main2():
    #Vamos a generar los datos como la tabla dada.
    data = pd.DataFrame(data2)
    # data.hist(bins = range(90,121,5))
    table, bins = create_freq_dist_table2(data)
    stats = data_from_table(table)
    d = pd.DataFrame({'': ['Media','Standard Deviation','Minimum','25%','50%','75%','Maximum','Skew','Kurtosis'],"data":stats})
    print(d)
    datos_generados = generate_data(table,10000)
    #data.hist(bins = range(90,121,5))
    #datos_generados.hist(bins = range(90,121,5))

    table2, bins = create_freq_dist_table2(datos_generados)
    stats2 = data_from_table(table2)
    d2 = pd.DataFrame({'': ['Media','Standard Deviation','Minimum','25%','50%','75%','Maximum','Skew','Kurtosis'],"data":stats2}).set_index('')
    media2 = d2.loc['Media'][0]
    std2 = d2.loc['Standard Deviation'][0]
    multados = media2 + 2*std2
    print(d2)
    multados = media2 + 2*std2
    print(multados)
    print("Hola")
main2()